In [1]:
import os
import torch
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available():
    os.environ['CUDA_VISIBLE_DEVICES']='0'
    torch.backends.cudnn.benchmark=True
    
print('Using device:', device)

Using device: cpu


In [2]:
from fasterai.visualize import *
plt.style.use('dark_background')

In [3]:
colorizer = get_video_colorizer()

# Instructions

### source_url
Type in a url hosting a video from YouTube, Imgur, Twitter, Reddit, Vimeo, etc.  Many sources work!  GIFs also work.  Full list here: https://ytdl-org.github.io/youtube-dl/supportedsites.html NOTE: If you want to use your own video, you can set source_url to None and just upload the file to video/source/ in Jupyter.  Just make sure that the file_name parameter matches the file you uploaded.


### file_name
Name this whatever sensible file name you want (minus extension)! It should actually exist in video/source if source_url=None


### render_factor
The default value of 21 has been carefully chosen and should work -ok- for most scenarios (but probably won't be the -best-). This determines resolution at which the color portion of the video is rendered. Lower resolution will render faster, and colors also tend to look more vibrant. Older and lower quality film in particular will generally benefit by lowering the render factor. Higher render factors are often better for higher quality videos and inconsistencies (flashy render) will generally be reduced, but the colors may get slightly washed out. 


### file_name_ext
There's no reason to changes this.


### result_path
Ditto- don't change.


### How to Download a Copy
Simply shift+right click on the displayed video and click "Save video as..."!


## Pro Tips
1. If a video takes a long time to render and you're wondering how well the frames will actually be colorized, you can preview how well the frames will be rendered at each render_factor by using the code at the bottom. Just stop the video rendering by hitting the stop button on the cell, then run that bottom cell under "See how well render_factor values perform on a frame here". It's not perfect and you may still need to experiment a bit especially when it comes to figuring out how to reduce frame inconsistency.  But it'll go a long way in narrowing down what actually works.


## Troubleshooting
The video player may wind up not showing up, in which case- make sure to wait for the Jupyter cell to complete processing first (the play button will stop spinning).  Then follow these alternative download instructions

1. In the menu to the left, click Home icon.
2. By default, rendered video will be in /video/result/

If a video you downloaded doesn't play, it's probably because the cell didn't complete processing and the video is in a half-finished state.
If you get a 'CUDA out of memory' error, you probably have the render_factor too high.  The max is 44 on 11GB video cards.

## Colorize!!

In [4]:
#NOTE:  Max is 44 with 11GB video cards.  21 is a good default
render_factor=21
#NOTE:  Make source_url None to just read from file at ./video/source/[file_name] directly without modification
source_url='https://twitter.com/silentmoviegifs/status/1116751583386034176'
file_name = 'DogShy1926'
file_name_ext = file_name + '.mp4'
result_path = None

if source_url is not None:
    result_path = colorizer.colorize_from_url(source_url, file_name_ext, render_factor=render_factor)
else:
    result_path = colorizer.colorize_from_file_name(file_name_ext)

show_video_in_notebook(result_path)

[twitter] 1116751583386034176: Downloading webpage
[twitter:card] 1116751583386034176: Downloading webpage
[twitter:card] 1116751583386034176: Downloading guest token
[twitter:card] 1116751583386034176: Downloading JSON metadata
[download] Destination: video/source/DogShy1926.mp4
[download] 100% of 227.58KiB in 00:0076MiB/s ETA 00:000


Video created here: video/result/DogShy1926.mp4


## See how well render_factor values perform on a frame here

In [ ]:
for i in range(10,45,2):
    colorizer.vis.plot_transformed_image('video/bwframes/' + file_name + '/00001.jpg', render_factor=i, display_render_factor=True, figsize=(8,8))